In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [2]:
import numpy as np
import struct
import os

# Load the MNIST data for this exercise
# mat_data contain the training and testing images or labels.
#   Each matrix has size [m,n] for images where:
#      m is the number of examples.
#      n is the number of pixels in each image.
#   or Each matrix has size [m,1] for labels contain the corresponding labels (0 to 9) where:
#      m is the number of examples.
def load_mnist(file_dir, is_images='True'):
    # Read binary data
    bin_file = open(file_dir, 'rb')
    bin_data = bin_file.read()
    bin_file.close()
    # Analysis file header
    if is_images:
        # Read images
        fmt_header = '>iiii'
        magic, num_images, num_rows, num_cols = struct.unpack_from(fmt_header, bin_data, 0)
    else:
        # Read labels
        fmt_header = '>ii'
        magic, num_images = struct.unpack_from(fmt_header, bin_data, 0)
        num_rows, num_cols = 1, 1
    data_size = num_images * num_rows * num_cols
    mat_data = struct.unpack_from('>' + str(data_size) + 'B', bin_data, struct.calcsize(fmt_header))
    mat_data = np.reshape(mat_data, [num_images, num_rows * num_cols])
    print('Load images from %s, number: %d, data shape: %s' % (file_dir, num_images, str(mat_data.shape)))
    return mat_data

# tranfer the image from gray to binary and get the one-hot style labels
def data_convert(x, y, m, k):
    x[x<=40]=0
    x[x>40]=1
    ont_hot_y = np.zeros((m,k))    #(60000,10)
    for t in np.arange(0,m):
        ont_hot_y[t,y[t]]=1
    ont_hot_y=ont_hot_y.T #(10,60000)
    return x, ont_hot_y

# call the load_mnist function to get the images and labels of training set and testing set
def load_data(mnist_dir, train_data_dir, train_label_dir, test_data_dir, test_label_dir):
    print('Loading MNIST data from files...')
    
    print(os.path.join(mnist_dir, train_data_dir))
    train_images = load_mnist(os.path.join(mnist_dir, train_data_dir), True)
    train_labels = load_mnist(os.path.join(mnist_dir, train_label_dir), False)
    test_images = load_mnist(os.path.join(mnist_dir, test_data_dir), True)
    test_labels = load_mnist(os.path.join(mnist_dir, test_label_dir), False)
    return train_images, train_labels, test_images, test_labels


In [3]:
mnist_dir = "./mnist_data/"
train_data_dir = "train-images.idx3-ubyte"
train_label_dir = "train-labels.idx1-ubyte"
test_data_dir = "t10k-images.idx3-ubyte"
test_label_dir = "t10k-labels.idx1-ubyte"

In [4]:
train_images, train_labels, test_images, test_labels = load_data(mnist_dir, train_data_dir, train_label_dir, test_data_dir, test_label_dir)
print("Got data. ") 

Loading MNIST data from files...
./mnist_data/train-images.idx3-ubyte
Load images from ./mnist_data/train-images.idx3-ubyte, number: 60000, data shape: (60000, 784)
Load images from ./mnist_data/train-labels.idx1-ubyte, number: 60000, data shape: (60000, 1)
Load images from ./mnist_data/t10k-images.idx3-ubyte, number: 10000, data shape: (10000, 784)
Load images from ./mnist_data/t10k-labels.idx1-ubyte, number: 10000, data shape: (10000, 1)
Got data. 


In [5]:
len(train_images)

60000

In [6]:
len(test_images)

10000

In [20]:
np.sqrt(len(test_images[0]))

28.0

In [ ]:
for i in range(2):
    img = np.reshape(train_images [i, :], (28, 28))
    label = np.argmax(train_images [i, :])
    plt.matshow(img, cmap = plt.get_cmap('gray'))
    plt.figure(figsize=(1,1))
    plt.show()


In [128]:
class Conv:
    def __init__(self, in_channels, out_channels, filter_size, stride=1, padding=0):
        """
        params: in_channels: the number of channels of the input image
                out_channels: the number of channels of the output image
                filter_size:(x,y) the size of the filter
                stride: the stride of the filter
                padding: the padding of the filter
        """
        self.input = None
        self.output = None
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.filter_size = filter_size
        self.stride = stride
        self.padding = padding
        self.Weight = np.random.normal(scale=1, size=(out_channels,in_channels,filter_size[0],filter_size[1]))
        #第一个维度是输出通道数，对应着有这么多个卷积核；第二个维度是输入通道数，这是因为卷积核需要负责讲in个channel的输入变成out个channel的输出

        self.W_grad = None
        self.Bias = np.zeros(out_channels) #每个卷积核有一个偏置参数
        self.B_grad = None
    
    def Conv2D(self):
        # def conv(x,y,i,j): #从(x,y)到(x+filter_size,t+filter_size)的input与weight[i,j]的卷积计算
        #     return np.sum(X[x:x+self.filter_size,y:y+self.filter_size] * self.Weight[i,j])
        
        # res = []
        # for i in range(self.in_channels):
        #     for o in range(self.out_channels):
        #         res.extend(joblib.Parallel(n_jobs=-1,verbose=0)(joblib.delayed(conv)(h*self.stride,w*self.stride,i,o) for h in range(self.output_H) for w in range(self.output_W) ))
        # return np.array(res).reshape(self.in_channels,self.out_channels,self.output_H,self.output_W)
        for i in range(self.in_channels):
            for o in range(self.out_channels):
                for h in range(self.output_H):
                    for w in range(self.output_W):
                        self.output[i,o,h,w] = np.sum(self.input[h*self.stride:h*self.stride+self.filter_size[0],
                                                                 w*self.stride:w*self.stride+self.filter_size[1]] * self.Weight[i,o])
                
    def forward(self, X):
        C,W,H = X.shape  
        # C for Channels, W for Width, H for Height

        # 进行padding操作，填充0
        if self.padding!= 0:
            for channels in range(C):
                X[channels] = np.pad(X[channels], ((self.padding, self.padding), (self.padding, self.padding)),
                                     'constant',constant_values = (0,0))
            
        output_W = (W + 2*self.padding - self.filter_size[0]) // self.stride + 1
        output_W = int(output_W)
        output_H = (H + 2*self.padding - self.filter_size[1]) // self.stride + 1
        output_H = int(output_H)
        self.output = np.zeros((self.out_channels, output_H, output_W))
        
        for o in range(self.out_channels):
                for h in range(output_H):
                    for w in range(output_W):
                        self.output[o,h,w] = np.sum(X[:,h*self.stride:h*self.stride+self.filter_size[0],
                                                                 w*self.stride:w*self.stride+self.filter_size[1]] * self.Weight[o])+self.Bias[o]
        return self.output
    
    def backprop(self, dout):
        pass

    

In [124]:
a = np.random.normal(size = (3,3,3))

In [102]:
class MaxPool:
    def __init__(self,pool_size=None,stride = 1):
        if pool_size is None:
            pool_size = [2, 2]
        self.pool_size = pool_size
        self.stride = stride
        self.output = None
    
    def forward(self, X):
        in_channels,h,w = X.shape
        output_h = h // self.pool_size[0]
        output_w = w // self.pool_size[1]
        self.output = np.zeros((in_channels,output_h,output_w))
        for i in range(output_h):
            for j in range(output_w):
                self.output[:,i,j] = np.max(X[:, i*self.pool_size[0]:(i+1)*self.pool_size[0], j*self.pool_size[1]:(j+1)*self.pool_size[1]], axis=(1, 2))
        return self.output
    
    def backprop(self, back_grad):
        pass

        


In [130]:
class Linear:
    def __init__(self,input_size,output_size) :
        self.Weight = np.random.normal(scale=1, size=(input_size, output_size))
        self.W_grad = None
        self.Bias = np.zeros(output_size)
        self.B_grad = None
    
    def forward(self,X):
        return np.dot(X,self.Weight) + self.Bias
    
    def back_prop(self,back_grad):
        pass
        

In [93]:
test = train_images[0]

In [17]:
len(test)

784

In [98]:
C1 = Conv(1, 6, (5,5))

test = test.reshape(1,28,28)
x = C1.forward(test)

In [97]:
x.shape

(6, 26, 26)

In [99]:
def ReLu(x):
    return np.maximum(0,x)

In [100]:
x = ReLu(x)

In [101]:
x.shape

(6, 24, 24)

In [103]:
S2 = MaxPool((2,2))
x = S2.forward(x)

In [104]:
x.shape

(6, 12, 12)

In [106]:
C3 = Conv(6,16,(5,5))
x = C3.forward(x)

In [108]:
x = ReLu(x)

In [111]:
S4 = MaxPool((2,2))
x = S4.forward(x)

In [129]:
x.shape

(16, 4, 4)

In [134]:
x1 = x.flatten()

In [135]:
C5 = Linear(16*4*4,120)
x1 = C5.forward(x1)

In [136]:
x1.shape

(120,)

In [137]:
F6 = Linear(120,84)

In [138]:
x1 = F6.forward(x1)

In [140]:
x1.shape

(84,)

In [141]:
output = Linear(84,10)
x1 = output.forward(x1)

In [143]:
x1.shape

(10,)

In [108]:
a = np.random.normal(scale=1, size=(1,6,5,5))
d = np.random.normal(scale=1,size=6)

In [68]:
t = np.random.normal(scale=1,size=(28,28))

In [83]:
def Conv2D():
        def conv(x,y,i,j): #从(x,y)到(x+filter_size,t+filter_size)的input与weight[i,j]的卷积计算
            return np.sum(t[x:x+5,y:y+5] * a[i,j])
        
     
        res = (joblib.Parallel(n_jobs=-1,verbose=0)(joblib.delayed(conv)(h,w,i,o)for i in range(1) for o in range(6) for h in range(24) for w in range(24) ))
        return np.array(res).reshape(1,6,24,24)
                


In [109]:
b = np.zeros(shape=(1,6,24,24))
for i in range(1):
    for j in range(6):
        for k in range(24):
            for l in range(24):
                b[i][j][k][l] = np.sum(t[k:k+5,l:l+5] * a[i,j])+d[j]

In [ ]:
t.tolist()

In [104]:
from scipy.signal import convolve2d
input_matrix = np.random.rand(28, 28)
convolution_kernel = np.random.rand(5, 5)

result = convolve2d(t, a[0,1], mode='valid')

In [96]:
result.shape

(24, 24)

In [97]:
result.sum()

121.53240548251058

In [100]:
b[0,1].sum()

115.38186510590201

In [101]:
c = Conv2D()

In [102]:
c[0,1].sum()

115.38186510590201

In [94]:
sum(b[0,1] - result)

array([ -3.93313814, -24.53926697,  -1.97669086,  35.7300471 ,
        22.27250686,  -0.64002107, -25.22907756, -23.95069935,
        -5.53545181,   4.35950017,  13.34553573,  23.17592675,
        -3.65901826, -30.33391001, -33.09603435,   6.08068901,
        36.29357402,  10.10776773,   4.87682599,  -9.26808911,
         7.09052633,   5.54335861,  -9.90899801,  -2.95640318])

In [25]:
a[0][1]

array([[ 0.34859123,  0.06920816,  2.27276008,  0.23363653,  0.22236938],
       [ 0.12490571,  1.29480527, -0.50622616, -1.66978477,  0.21479421],
       [-1.31754901, -1.13363826,  1.61447842,  0.3093921 ,  0.83311412],
       [-0.12174634,  1.71252122, -0.40290676,  1.70859997,  0.19225454],
       [-0.73175619, -0.73192317, -0.83354696,  0.8423247 ,  0.18075662]])

In [49]:
a[0,2,:,:]

array([[-5.20685966e-01,  1.92357766e-01, -6.75320741e-01,
        -5.50791318e-01,  1.25817385e+00],
       [ 4.39842884e-01, -1.00497408e+00,  7.82645803e-01,
        -3.32227220e-01,  1.24518928e+00],
       [-1.48506636e+00, -1.74401920e+00,  6.14466219e-01,
         1.01814999e+00, -1.90725948e+00],
       [-1.43050082e-01, -5.42540574e-01,  1.89386318e+00,
         4.71446098e-01, -2.75087901e-01],
       [ 7.34852903e-01, -1.31681826e+00,  2.49603101e+00,
         1.33807771e+00, -1.42505639e-03]])

In [65]:
a[0,2,:,:]

array([[-5.20685966e-01,  1.92357766e-01, -6.75320741e-01,
        -5.50791318e-01,  1.25817385e+00],
       [ 4.39842884e-01, -1.00497408e+00,  7.82645803e-01,
        -3.32227220e-01,  1.24518928e+00],
       [-1.48506636e+00, -1.74401920e+00,  6.14466219e-01,
         1.01814999e+00, -1.90725948e+00],
       [-1.43050082e-01, -5.42540574e-01,  1.89386318e+00,
         4.71446098e-01, -2.75087901e-01],
       [ 7.34852903e-01, -1.31681826e+00,  2.49603101e+00,
         1.33807771e+00, -1.42505639e-03]])

In [66]:
a[0,2,1:3,:2]

array([[ 0.43984288, -1.00497408],
       [-1.48506636, -1.7440192 ]])

In [119]:
b = [
    [1,2],
    [3,4]
]
b = np.array(b)

In [122]:
b.sum(axis=(0,1))

10

In [58]:
np.sum(a[0,2,:2,:2] * b)

-2.1578606973703502

In [53]:
b

array([[1, 2],
       [3, 4]])

In [59]:
import joblib

def conv(x,y):
    return np.sum(a[0,2,x:x+2,y:y+2] * b)
res = joblib.Parallel(n_jobs=-1,verbose=2)(joblib.delayed(conv)(x,y) for x in range(5-2+1) for y in range(5-2+1))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  16 | elapsed:    1.0s remaining:    4.8s
[Parallel(n_jobs=-1)]: Done  12 out of  16 | elapsed:    1.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    1.1s finished


In [60]:
res

[-2.1578606973703502,
 2.6552983914667734,
 -1.2394024457623216,
 -0.2315486671161362,
 1.1858641247199175,
 0.570865537147406,
 1.5185567974674905,
 -4.502825976596664,
 -0.14053765221159542,
 0.07791833080551491,
 -1.8261008551351605,
 2.1788754736839815,
 -1.6521806703091342,
 1.376445515390394,
 0.26446378283225935,
 -0.5929687680394632]

In [63]:
np.array(res).reshape(4,)

array([[-2.1578607 ,  2.65529839, -1.23940245, -0.23154867],
       [ 1.18586412,  0.57086554,  1.5185568 , -4.50282598],
       [-0.14053765,  0.07791833, -1.82610086,  2.17887547],
       [-1.65218067,  1.37644552,  0.26446378, -0.59296877]])